In [ ]:
 import numpy as np
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 16525, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 16525 (delta 0), reused 2 (delta 0), pack-reused 16522
Receiving objects: 100% (16525/16525), 15.05 MiB | 11.54 MiB/s, done.
Resolving deltas: 100% (11353/11353), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.6/749.6 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 

In [ ]:
# Montar Google Drive en Colab
from google.colab import drive
drive.mount('/content/drive')

# Ruta a la carpeta en Google Drive
folder_path = '/content/drive/MyDrive/YOLOV5/'
!unzip -q /content/drive/MyDrive/YOLOV5/datacopia.zip -d /content/

Mounted at /content/drive


In [ ]:
import os
import yaml
import subprocess
import torch.nn as nn
import torch.nn.functional as F
import math

# Define la función autopad
def autopad(k, p=None, d=1):
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]  # actual kernel-size
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p

# Define algunas funciones de activación adicionales
class FReLU_noBN_biasFalse(nn.Module):
    def __init__(self, c2):
        super().__init__()
        self.frelu = nn.ReLU()
    def forward(self, x):
        return self.frelu(x)

# Define la clase Conv con opciones para configurar la función de activación
class Conv(nn.Module):
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p), groups=g, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        if act is True:
            self.act = nn.Identity()
        elif isinstance(act, nn.Module):
            self.act = act
        elif act == "sigmoid":
            self.act = nn.Sigmoid()
        elif act == "leakyrelu":
            self.act = nn.LeakyReLU()
        elif act == "silu":
            self.act = nn.SiLU()
        else:
            self.act = nn.Identity()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

# Código para cambiar la tasa de aprendizaje en un archivo YAML
def change_lr(yaml_file, new_lr):
    with open(yaml_file, 'r') as f:
        config = yaml.safe_load(f)
    config['lr0'] = new_lr
    config['lrf'] = 0.1 * new_lr
    with open(yaml_file, 'w') as f:
        yaml.dump(config, f)

# Ruta al archivo YAML
yaml_file = '/content/yolov5/data/hyps/hyp.scratch-high.yaml'

# Vector con diferentes valores de new_lr
new_lr_values = [0.001]

# Lista de funciones de activación a probar
activation_functions = ["silu"]

# Ruta base para los pesos
base_weights_path = '/content/yolov5/runs/train/'

# VOC
for new_lr in new_lr_values:
    for activation_function in activation_functions:
        for i, (b, m) in enumerate(zip([16], ['Adam'])):
            # Cambiar la tasa de aprendizaje
            change_lr(yaml_file, new_lr)

            # Construir y ejecutar el comando de entrenamiento
            train_command = f"python train.py --batch {b} --weights yolov5m.pt  --data /content/drive/MyDrive/YOLOV5/custom22.yaml --epochs 1000 --cache --img 640  --hyp hyp.scratch-high.yaml --optimizer {m} --learning_rate {new_lr} --activation {activation_function}"
            print(train_command)  # Imprime el comando que se ejecutará
            # subprocess.run(train_command, shell=True)  # Ejecutar el comando de entrenamiento
            #subprocess.run(train_command.split())
            !python train.py --batch {b} --weights yolov5m.pt  --data /content/drive/MyDrive/YOLOV5/custom22.yaml --epochs 1000 --cache --img 640  --hyp hyp.scratch-high.yaml --optimizer {m}

            # Construir la ruta para los pesos
            exp_folder = f"exp{i+1}" if i > 0 else "exp"  # Ajuste la indexación aquí
            weights_path = os.path.join(base_weights_path, f"{exp_folder}/weights/last.pt")

            # Construir y ejecutar el comando de validación
            val_command = f"python val.py - --data /content/drive/MyDrive/YOLOV5/custom22.yaml --weights {weights_path} --img 640 --batch_size {b}"
            print(val_command)  # Imprime el comando que se ejecutará
            # subprocess.run(val_command, shell=True)  # Ejecutar el comando de validación
            #subprocess.run(val_command.split())
            !python val.py --batch {b}  --data /content/drive/MyDrive/YOLOV5/custom22.yaml --weights {weights_path} --img 640


python train.py --batch 16 --weights yolov5m.pt  --data /content/drive/MyDrive/YOLOV5/custom22.yaml --epochs 1000 --cache --img 640  --hyp hyp.scratch-high.yaml --optimizer Adam --learning_rate 0.001 --activation silu
2024-04-08 04:53:57.846098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 04:53:57.846151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 04:53:57.847617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=, data=/content/drive/MyDrive/YOLOV5/custom22.yaml, hyp=hyp.scratch-high.yaml, epochs=1000, batch_size=16, imgsz=640

In [ ]:
!python val.py --data  /content/drive/MyDrive/YOLOV5/custom22.yaml --weights /content/yolov5/runs/train/exp2/weights/best.pt  --img 640

val: data=/content/drive/MyDrive/YOLOV5/custom22.yaml, weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-296-gae4ef3b2 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning /content/datacopia/labels/val.cache... 33 images, 0 backgrounds, 0 corrupt: 100% 33/33 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 2/2 [00:03<00:00,  1.73s/it]
                   all         33        290      0.988      0.995      0.993      0.861
               limones         33        164      0.977          1      0.991      0.82

In [ ]:
!python val.py --data /content/drive/MyDrive/YOLOV5/custom22.yaml --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 640 --task test

val: data=/content/drive/MyDrive/YOLOV5/custom22.yaml, weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-296-gae4ef3b2 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs
test: Scanning /content/datacopia/labels/test.cache... 14 images, 0 backgrounds, 0 corrupt: 100% 14/14 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 1/1 [00:01<00:00,  1.82s/it]
                   all         14        161      0.965      0.967       0.99       0.56
               limones         14         89      0.989      0.976      0.991      0

In [ ]:
import shutil
import os

# Ruta de la carpeta que deseas comprimir
folder_path = '/content/yolov5/runs'

# Nombre del archivo ZIP que se creará
zip_filename = 'train,val,test,1000,M.zip'

# Crear el archivo ZIP
shutil.make_archive(zip_filename, 'zip', folder_path)

# Mover el archivo ZIP al directorio de descarga
shutil.move(zip_filename + '.zip', '/content')

# Imprimir mensaje de confirmación
print(f"El archivo ZIP {zip_filename} se ha creado y descargado correctamente.")

El archivo ZIP train,val,test,1000,M.zip se ha creado y descargado correctamente.
